In [1]:
import json
import graphviz
import csv
import numpy as np
import tifffile

In [2]:
annotation_file_2017_32bit = '/jukebox/LightSheetTransfer/atlas/allen_atlas/annotation_2017_25um_sagittal_forDVscans.nrrd'
annotation_file_2017_16bit = 'annotation_2017_25um_sagittal_16bit.tif'

## Purpose
This notebook is where I will try to make simpler annotation volumes of the Allen atlas using the ontology structure graph.

## First convert atlas to 16 bit

In [3]:
vol_32bit = np.array(tifffile.imread(annotation_file_2017_32bit),dtype='uint32')

In [137]:
vol_32bit.dtype

dtype('uint32')

In [138]:
segments_32bit = np.unique(vol_32bit)

In [139]:
segments_32bit

array([        0,         1,         2,         6,         7,         9,
              10,        12,        15,        17,        19,        20,
              23,        26,        27,        28,        30,        33,
              35,        36,        38,        41,        42,        50,
              52,        54,        56,        58,        59,        62,
              63,        64,        66,        67,        68,        72,
              74,        75,        78,        81,        83,        84,
              88,        91,        93,        96,        97,        98,
             100,       101,       102,       105,       106,       108,
             113,       114,       115,       117,       118,       120,
             121,       122,       123,       125,       126,       128,
             129,       131,       132,       133,       136,       139,
             140,       143,       145,       146,       147,       148,
             149,       153,       155,       156, 

In [150]:
2**32-1

4294967295

In [158]:
vol_16bit = vol_32bit.astype('uint16')
# vol_16bit = np.array(tifffile.imread(annotation_file_2017_32bit),dtype='uint16')

In [159]:
segments_16bit = np.unique(vol_16bit)

In [160]:
segments_16bit

array([    0,     1,     2,     6,     7,     9,    10,    12,    15,
          17,    19,    20,    23,    26,    27,    28,    30,    33,
          35,    36,    38,    41,    42,    50,    52,    54,    56,
          58,    59,    62,    63,    64,    66,    67,    68,    72,
          74,    75,    78,    81,    83,    84,    88,    91,    93,
          96,    97,    98,   100,   101,   102,   105,   106,   108,
         113,   114,   115,   117,   118,   120,   121,   122,   123,
         125,   126,   128,   129,   131,   132,   133,   136,   139,
         140,   143,   145,   146,   147,   148,   149,   153,   155,
         156,   158,   159,   162,   163,   164,   169,   171,   173,
         177,   178,   180,   181,   186,   187,   188,   189,   190,
         194,   196,   197,   198,   201,   202,   203,   204,   206,
         207,   209,   210,   211,   214,   215,   217,   218,   222,
         223,   225,   226,   229,   230,   231,   233,   234,   237,
         238,   243,

In [161]:
len(segments_32bit)

619

In [162]:
len(segments_16bit)

618

So unfortunately it looks like there was one value in the 32 bit image that maps to a value already in the volume when converted to 16 bit. Let's find which one

In [163]:
for segment_32 in segments_32bit:
    print(segment_32,np.array(segment_32).astype('uint16'))

0 0
1 1
2 2
6 6
7 7
9 9
10 10
12 12
15 15
17 17
19 19
20 20
23 23
26 26
27 27
28 28
30 30
33 33
35 35
36 36
38 38
41 41
42 42
50 50
52 52
54 54
56 56
58 58
59 59
62 62
63 63
64 64
66 66
67 67
68 68
72 72
74 74
75 75
78 78
81 81
83 83
84 84
88 88
91 91
93 93
96 96
97 97
98 98
100 100
101 101
102 102
105 105
106 106
108 108
113 113
114 114
115 115
117 117
118 118
120 120
121 121
122 122
123 123
125 125
126 126
128 128
129 129
131 131
132 132
133 133
136 136
139 139
140 140
143 143
145 145
146 146
147 147
148 148
149 149
153 153
155 155
156 156
158 158
159 159
162 162
163 163
164 164
169 169
171 171
173 173
177 177
178 178
180 180
181 181
186 186
187 187
188 188
189 189
190 190
194 194
196 196
197 197
198 198
201 201
202 202
203 203
204 204
206 206
207 207
209 209
210 210
211 211
214 214
215 215
217 217
218 218
222 222
223 223
225 225
226 226
229 229
230 230
231 231
233 233
234 234
237 237
238 238
243 243
246 246
249 249
250 250
251 251
252 252
255 255
257 257
258 258
260 260
262 262
263 

Looks like value 527696992 is the culprit. If we just change this value to something else then try mapping again:

In [164]:
mask = vol_32bit == 527696992
vol_32bit[mask] = 527696992 + 1000

Now let's try converting again:

In [168]:
segments_32bit = np.unique(vol_32bit)

In [169]:
len(segments_32bit)

619

In [165]:
vol_16bit = vol_32bit.astype('uint16')

In [5]:
segments_16bit = np.unique(vol_16bit)

NameError: name 'vol_16bit' is not defined

In [4]:
len(segments_16bit)

NameError: name 'segments_16bit' is not defined

OK, so it should have worked and now we have a proper 16 bit image which has the same volume structure, only some IDs have changed

In [170]:
set_32 = set(segments_32bit)
set_16 = set(segments_16bit)

In [171]:
set_32.difference(set_16)

{182305696,
 182305712,
 312782560,
 312782592,
 312782624,
 312782656,
 484682464,
 484682496,
 484682528,
 496345664,
 526157184,
 526322272,
 527697992,
 549009216,
 560581568,
 563807424,
 576073728,
 589508416,
 589508480,
 599626944,
 606826624,
 606826688,
 607344832,
 614454272}

In [172]:
set_16.difference(set_32)

{1216,
 2120,
 2656,
 12096,
 12160,
 12288,
 14144,
 22720,
 28800,
 28864,
 34176,
 38080,
 41536,
 43744,
 43776,
 43808,
 44768,
 44800,
 44832,
 44864,
 50080,
 50096,
 52160,
 54272}

In [173]:
len(set_32.difference(set_16))

24

## Updating the ontology JSON file for 16 bit

In [174]:
len(set_16.difference(set_32))

24

So the Ids we need to replace in the JSON file are the following:

In [178]:
set_32_diff16 = sorted(list(set_32.difference(set_16)))
set_16_diff32 = sorted(list(set_16.difference(set_32)))
print("# 32bit 16bit")
for ii in range(len(set_32_diff16)):
    print(set_32_diff16[ii],set_16_diff32[ii])

# 32bit 16bit
182305696 1216
182305712 2120
312782560 2656
312782592 12096
312782624 12160
312782656 12288
484682464 14144
484682496 22720
484682528 28800
496345664 28864
526157184 34176
526322272 38080
527697992 41536
549009216 43744
560581568 43776
563807424 43808
576073728 44768
589508416 44800
589508480 44832
599626944 44864
606826624 50080
606826688 50096
607344832 52160
614454272 54272


Only a few of the 32 bit ids were actually in the JSON file. Saved the new JSON file as:
```/home/ahoag/Git/lightserv_dj_demo/graphviz/allen_ontology_16bit.json ```

In [179]:
# Save the 16 bit volume
tifffile.imsave('./annotation_2017_25um_sagittal_16bit.tif',vol_16bit,dtype='uint16')

## Now save the segments to a file

In [7]:
vol_16bit = np.array(tifffile.imread('annotation_2017_25um_sagittal_16bit.tif'),dtype='uint16')

In [10]:
segments_16bit = np.unique(vol_16bit)

In [14]:
np.savetxt('annotation_2017_25um_sagittal_16bit_segments.txt',segments_16bit,fmt="%i")